In [30]:
from data_parser import VelParser
from entropy_compressor import HuffmanCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import os
import shutil
import numpy as np
from typing import Dict, List
from pathlib import Path
from pprint import pprint

COMPRESSED_PATH = 'compressed'


class Compressor:
    def __init__(self):
        self.entropy_compressor = HuffmanCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
        self.parser = VelParser()
    
    def total_compress(self, data: np.ndarray) -> List[np.ndarray | np.float64]:
        wavelet_compressed = self.wavelet_compressor.wavelet_transform(data)[0]
        quantized, q_step, min_val = self.quantizator.uniform_quantization(wavelet_compressed)
        entropy_compressed = self.entropy_compressor.huffman_encode_ndarray(quantized)
        return [entropy_compressed, q_step, min_val]

    def save_mesh(self, mesh_data: Dict[str, np.ndarray]) -> None:
        # faces, boundary, neighbour, owner, pts = mesh_data.values()
        with open(COMPRESSED_PATH + '/' + 'mesh', mode='w+') as f:
            print(list(mesh_data.keys())[:-1])
            for (key, value) in list(mesh_data.items())[:-1]:
                compressed_ndarray, q_step, min_val = self.total_compress(value)
                f.write(f'{q_step} {min_val}\n')
                f.write(str(compressed_ndarray) + '\n')

    def compress_and_save(self, model: str, base_path: str, dim: str):
        vels = list(map(str, sorted(self.parser.get_vels(
            model=model,
            base_path=base_path,
            dimpath=dim,
        ), key=lambda x: str(x))))
        mesh_data = self.parser.get_mesh_data(
            path_to_vel=vels[0],
        )
        print(mesh_data.keys())
        self.save_mesh(mesh_data=mesh_data)

        for vel in vels:
            data = self.parser.get_vel_data(
                path_to_vel=vel,
            )
            vellast = vel.split('/')[-1]
            with open(COMPRESSED_PATH + '/' + vellast, mode='w+') as f:
                times = list(data.keys())
                f.write(f'{";".join(times)}\n')
                for (_, value) in list(data.items())[1:]:
                    p, t, u, rho = value
                    compressed_p, p_q_step, p_min_val = self.total_compress(p)
                    compressed_t, t_q_step, t_min_val = self.total_compress(t)
                    compressed_u, u_q_step, u_min_val = self.total_compress(u)
                    compressed_rho, rho_q_step, rho_min_val = self.total_compress(rho)
                    f.write(f'{p_q_step} {p_min_val}\n')
                    f.write(f'{str(compressed_p)}' + '\n')
                    f.write(f'{t_q_step} {t_min_val}\n')
                    f.write(f'{str(compressed_t)}' + '\n')
                    f.write(f'{u_q_step} {u_min_val}\n')
                    f.write(f'{str(compressed_u)}' + '\n')
                    f.write(f'{rho_q_step} {rho_min_val}\n')
                    f.write(f'{str(compressed_rho)}' + '\n')
    
    def copy_not_compressable(self, model: str, base_path: str, dim: str):
        vels = list(map(str, sorted(self.parser.get_vels(
            model=model,
            base_path=base_path,
            dimpath=dim,
        ), key=lambda x: str(x))))
        zero_vel = vels[0]
        thermophysical_path = f'{zero_vel}/constant/thermophysicalProperties'
        turbulenceProperties = f'{zero_vel}/constant/turbulenceProperties'
        system_path = f'{zero_vel}/system'
        destination_path = f'compressed/not_compressable'
        # os.mkdir(path=destination_path)
        shutil.copytree(system_path, destination_path)
        shutil.copy(thermophysical_path, destination_path)
        shutil.copy(turbulenceProperties, destination_path)

In [ ]:
compressor = Compressor()
data = compressor.compress_and_save(
    model='data_wage',
    base_path='data',
    dim='low_dim',
)

In [31]:
compressor = Compressor()
compressor.copy_not_compressable(
    model='data_wage',
    base_path='data',
    dim='low_dim'
)

FileExistsError: [Errno 17] File exists: 'compressed/not_compressable'

In [ ]:
vels = list(map(str, sorted(compressor.parser.get_vels(
    model='data_wage',
    base_path='data',
    dimpath='low_dim',
), key=lambda x: str(x))))
mesh_data = compressor.parser.get_mesh_data(
    path_to_vel=vels[0],
)

In [28]:
compressor.total_compress(mesh_data['faces'])

['11001010101010111110010100111101011010101100110011000110111101101110110100100011100001100111111110001111110110110101100000110110101010111000011111100110000001011011101011100100111010100011001111011100100111100011000101011111011010001111100010111101001000101101011101100110010100011110001010000110110111011000011010100101100001010110001001011101001111001011100100100000001110001111011001000101110101010100111001000011110101011101011001010000101010000111101100000011001110010011011010110100010111000100010001001110100001010110100000001111001100101100111011000110101001011101100000101101001110111110010110001101001011001000011001100110011101011010100100110111110101000011111110100111111100000101011110101001111001000110110110111101111001000000000010100001011000111010001101000000011101000000101110011011111110000011110100110011010011100110101111001111000100101110100100101111111101110111001011111101111111100011101001111000010011000000010101110011111111111000001001101000010010100111111101010001110011

In [29]:
compressor.total_compress(mesh_data['neighbour'])

['111000010111111001111101011101110101101110011011110011111111110100010011110100110110010001111001011001011110100110111011111101111011101011110110010011011011111010001101010100001110100110001011100011100100111110011101011001001110111111100011101101100011110011000001100001110011111010101010110110111111010000000000000000000000000000000000000',
 0.002994060516357422,
 -30.03125]

In [30]:
compressor.total_compress(mesh_data['owner'])

['11110010110001111000111101100000111111101100011101001110011111101010000000010011100001110101011000101111111001110110101111001100010101110100011011110111101001000100010001011000001101000111001111110101011011100000101111111101110101011001100000100101100111111001011100100101010001101001100011010011001110111001001110111011011100010001010000100001110110000011110100010011100010011100101110000011101100100101101010001110110011011011011110000110001111111011001010011010010000110001011111101010111111110011100010111100111001100101001011101011',
 0.002654552459716797,
 -13.15625]

In [31]:
compressor.total_compress(mesh_data['points'])

['00110011000011000110000110001111000100011111010010110010000100000110000001111100001011100000111001001010100100100011110011011101010111010001101100000000110111001111101101001010011000110011110010110101011111100001100101110111111000101101111111100010001100000010010001111000100110011101001111001101011011101011101010110100011111111111111110010011111110100100110010001001101010011000110011111001100011011111000111011000011011100011110001110011111111010011101011011011110000111011111101011001011111011100110010101001101011001011100001101010001111011111100010010001010010110011000000001101111011110111101101100100011110010110011100011011001101100100100111011011001101011111001000010100001111001011010111010100111001000001011010011110110011111110101001101000110011100100110111110010000001010100011010001010111101000010110010001000000010101001110010111000100010000111011100010111110011011111101110111111101110001010100011000011101011001111010111110100011101100110011110110110011010101010110101001100111100